# JETARM - Live Demo

In [74]:
import torch
device = torch.device('cuda')

Load the optimized model.

In [75]:
import torchvision
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('model_trt.pth'))

<All keys matched successfully>

Create the preprocessing function.

In [76]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Start and display the camera.

In [77]:
import traitlets
from IPython.display import display
import ipywidgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = ipywidgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(ipywidgets.VBox([image]))

Create the robot instance to drive the motors.

In [78]:
from jetbot import Robot
from SCSCtrl import TTLServo
import time

robot = Robot()

def stop(change):
    robot.stop()

def limitCtl(maxInput, minInput, rawInput):
    if rawInput > maxInput:
        limitBuffer = maxInput
    elif rawInput < minInput:
        limitBuffer = minInput
    else:
        limitBuffer = rawInput
    return limitBuffer


gripPos = -50
gripMin = -90
gripMax = -10

xPos = 100
xMin = 140
xMax = 210

yPos = 0
yMin = -120
yMax = 120

def setGrip(newGripPos):
    global gripPos
    gripPos = limitCtl(gripMax, gripMin, newGripPos)
    TTLServo.servoAngleCtrl(4, gripPos, 1, 500)

def setXY(newXPos, newYPos):
    global xPos, yPos
    xPos = limitCtl(xMax, xMin, newXPos)
    yPos = limitCtl(yMax, yMin, newYPos)
    TTLServo.xyInput(xPos, yPos)

def setY(newYPos):
    global xPos, yPos
    yPos = limitCtl(yMax, yMin, newYPos)
    TTLServo.xyInput(xPos, yPos)

# Set the start position
# Turn the arm to the left
TTLServo.servoAngleCtrl(1, 90, 1, 150)
time.sleep(3)
TTLServo.servoStop(1)

# Extend the arm forward
setXY(xPos, yPos)

Create the update function which will pre-process the camera image, execute the nural network, and handle the results.

In [79]:
import torch.nn.functional as F
import time

prediction_widget = ipywidgets.widgets.Image(format='jpeg', width=camera.width, height=camera.height)

def update(change):
    global prediction_widget
    new_image = change['new'] 
    preprocessed = preprocess(new_image)
    output = model_trt(preprocessed).detach().cpu().numpy().flatten()

    # Thumb coordinates
    thumb_x = output[2 * 0]
    thumb_y = output[2 * 0 + 1]
    thumb_x = int(camera.width * (thumb_x / 2.0 + 0.5))
    thumb_y = int(camera.height * (thumb_y / 2.0 + 0.5))

    # Index coordinates
    index_x = output[2 * 1]
    index_y = output[2 * 1 + 1]
    index_x = int(camera.width * (index_x / 2.0 + 0.5))
    index_y = int(camera.height * (index_y / 2.0 + 0.5))

    # Set grip width
    grip_width = abs(thumb_x - index_x)
    new_grip_val = (2*grip_width) - 110
    new_grip_pos = limitCtl(gripMax, gripMin, new_grip_val)
    setGrip(new_grip_pos)

    # Set arm elivation
    hand_min_x = min([thumb_x, index_x])
    hand_max_x = max([thumb_x, index_x])
    hand_min_y = min([thumb_y, index_y])
    hand_max_y = max([thumb_y, index_y])

    hand_y_center = sum([hand_min_y, hand_max_y]) / 2
    new_hand_val = (0-hand_y_center) + 172
    new_hand_pos = limitCtl(yMax, yMin, new_hand_val)
    setY(new_hand_pos)

    # Move robot wheels
    if hand_min_x < 60 and hand_max_x < 164:
        if hand_min_x < 24:
            robot.forward(0.4)
        else:
            robot.forward(0.2)
    elif hand_max_x > 164 and hand_min_x > 60:
        if hand_min_x > 200:
            robot.backward(0.4)
        else:
            robot.backward(0.2)
    else:
        robot.stop()

    # Show hand tracking
    prediction = new_image.copy()
    prediction = cv2.circle(prediction, (thumb_x, thumb_y), 8, (255, 0, 0), 2)
    prediction = cv2.circle(prediction, (index_x, index_y), 8, (255, 0, 0), 2)
    cv2.rectangle(prediction, pt1=(48, 48), pt2=(176, 176), color=(0, 255, 255), thickness=1)
    
    # Show grip data
    grip_info = "W: " + str(grip_width) + ", P: " + str(new_grip_pos)
    thumb_info = "Th: " + str(thumb_x) + ", " + str(thumb_y)
    index_info = "In: " + str(index_x) + ", " + str(index_y)
    cv2.putText(prediction, grip_info, (32, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (51, 255, 153), 1, cv2.LINE_AA)
    cv2.putText(prediction, thumb_info, (32, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (51, 255, 153), 1, cv2.LINE_AA)
    cv2.putText(prediction, index_info, (32, 190), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (51, 255, 153), 1, cv2.LINE_AA)
    
    # Show elivation data
    elivation_center_info = "C: " + str(hand_y_center)
    elivation_pos_info = "HP: " + str(new_hand_pos)
    cv2.putText(prediction, elivation_center_info, (32, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (51, 255, 153), 1, cv2.LINE_AA)
    cv2.putText(prediction, elivation_pos_info, (32, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (51, 255, 153), 1, cv2.LINE_AA)
    
    # Show result
    prediction_widget.value = bgr8_to_jpeg(prediction)

# Display AI Camera feed with annotations
display(prediction_widget)

# Initialize
update({'new': camera.value})

Image(value=b'', format='jpeg', height='224', width='224')

Attach the camera to the nural network execution function.

In [80]:
camera.observe(update, names='value')

## End Demo

Unattach the callback function.

In [81]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)

robot.stop()

Close the camera conneciton so that it can be used later.

In [82]:
camera.stop()